# 抽取文档主题

## 什么是主题模型

### LDA，Latent Dirichlet Allocation

* Q: 有这么一篇文章，里面提到了詹姆斯、湖人队、季后赛，请问这篇文章最可能的主题是什么？

    * 军事
    * 体育
    * 养生
    * 教育
    
### LDA由Blei于2003年提出，其基本思想是把文档看成各种隐含主题的混合，而每个主题则表现为跟该主题相关的词项的概率分布

* 该方法不需要任何关于文本的背景知识
* 隐含主题的引入使得分析者可以对“一词多义”和“一义多词”的语言现象进行建模，更接近人类语言交互的特征

### LDA基于词袋模型构建，认为文档和单词都是可交换的，忽略单词在文档中的顺序和文档在语料库中的顺序，从而将文本信息转化为易于建模的数字信息

* 主题就是一个桶，里面装了出现概率较高的单词，这些单词与这个主题有很强的的相关性

## LDA模型包含词项、主题和文档三层结构

### 本质上，LDA简单粗暴的认为：文章中的每个词都是通过“以一定概率选择某个主题，再从该主题中以一定概率选择某个词”得到的

### 一个词可能会关联很多主题，因此需要计算各种情况下的概率分布，来确定最可能出现的主题是哪种

* 体育：{姚明：0.3，篮球：0.5，拳击：0.2，李现：0.03，王宝强：0.03，杨紫：0.04}
* 娱乐：{姚明：0.03，篮球：0.03，足球：0.04，李现：0.6，王宝强：0.7，杨紫：0.8}

### 一篇文章可能会涉及到几个主题，因此也需要计算多个主题的概率

* 体育新闻：\[废话，体育，体育，体育，....，娱乐，娱乐\]
* 八卦消息：\[废话，废话，废话，废话，....，娱乐，娱乐\]

## LDA中涉及到的数学知识

### 多项式分布：主题和词汇的概率分布服从多项式分布

* 如果1个词汇主题，就是大家熟悉的二项分布

### Dirichlet分布：上述多项式分布的参数为随机变量，均服从Dirichlet分布

### Gibbs抽样：直接求LDA的精确参数分布计算量太大，实际上不可行，因此通过Gibbs抽烟减小计算量，得到逼近的结果

* 通过现有文章（已有主题，或者需要提取主题）训练处LDA模型
* 用模型预测新的文章所属主题分类

### 主题模型对于  ***短文本***  效果不好



## 主题模型的sklearn实现

在scikit-learn中，LDA主题模型的类被放置在sklearn.decomposition.LatentDirichletAllocation类中，其算法实现主要基于变分推断EM算法，而没有使用基于Gibbs采样的MCMC算法实现。

注意由于LDA是基于词频统计的，因此理论上一般不宜用TF-IDF来做文档特征，但并非不能尝试。实际分析中也确实会见到此类操作。

> class sklearn.decomposition.LatentDirichletAllocation(
> 
> n_components = None : 隐含主题数K，需要设置的最重要参数。
    K的设定范围和具体的研究背景有关。
    K越大，需要的文档样本越多。
> 
> doc_topic_prior = None : 文档主题先验Dirichlet分布的参数α，未设定则用1/K。
> 
> topic_word_prior = None : 主题词先验Dirichlet分布的参数η，未设定则用1/K。
> 
> learning_method = 'online' : 即LDA的求解算法。'batch' | 'online'
    batch: 变分推断EM算法，会将将训练样本分批用于更新主题词分布，新版默认算法。
        样本量不大只是用来学习的话用batch比较好，这样可以少很多参数要调。
        需注意n_components(K), doc_topic_prior(α), topic_word_prior(η)
    online: 在线变分推断EM算法，大样本时首选。
        需进一步注意learning_decay, learning_offset，
            total_samples和batch_size等参数。
> 
> 仅在online算法时需要设定的参数
>
> learning_decay = 0.7 ：控制"online"算法的学习率，一般不用修改。
        取值最好在(0.5, 1.0]，以保证"online"算法渐进的收敛。
>
> learning_offset = 10. ：用来减小前面训练样本批次对最终模型的影响。
        取值要大于1。
>
>total_samples = 1e6 ： 分步训练时每一批文档样本的数量。
         使用partial_fit进行模型拟合时才需要此参数。
>
> batch_size = 128 : 每次EM算法迭代时使用的文档样本的数量。
> 
> )

***将语料库转换为所需矩阵***

除直接使用分词清理后文本进行转换外，也可以先计算关键词的TF-IDF值，然后使用关键词矩阵进行后续分析。

In [4]:
import pandas as pd
# 有的环境配置下read_table出错，因此改用read_csv
raw = pd.read_csv("金庸-射雕英雄传txt精校版.txt",
                  names = ['txt'], sep ='aaa', encoding ="GBK" ,engine='python')
# 章节判断用变量预处理
def m_head(tmpstr):
    return tmpstr[:1]

def m_mid(tmpstr):
    return tmpstr.find("回 ")

raw['head'] = raw.txt.apply(m_head)
raw['mid'] = raw.txt.apply(m_mid)
raw['len'] = raw.txt.apply(len)
# raw['chap'] = 0
raw.head(0)
# 章节判断
chapnum = 0
for i in range(len(raw)):
    if raw['head'][i] == "第" and raw['mid'][i] > 0 and raw['len'][i] < 30 :
        chapnum += 1
    if chapnum >= 40 and raw['txt'][i] == "附录一：成吉思汗家族" :
        chapnum = 0
    raw.loc[i, 'chap'] = chapnum
    
# 删除临时变量
del raw['head']
del raw['mid']
del raw['len']
raw.head(0)
#提取章节
rawgrp = raw.groupby('chap')
chapter = rawgrp.sum()##.agg(sum) # 只有字符串列的情况下，sum函数自动转为合并字符串
chapter = chapter[chapter.index != 0]

In [5]:
# 设定分词及清理停用词函数
# 熟悉Python的可以使用 open('stopWord.txt').readlines（） 获取停用词list，效率更高
stoplist = list(pd.read_csv('停用词.txt', names = ['w'], sep = 'aaa', 
                            encoding = 'utf-8', engine='python').w)
import jieba 
def m_cut(intxt):
    return [ w for w in jieba.cut(intxt) 
            if w not in stoplist and len(w) > 1 ] 

In [6]:
# 生成分词清理后章节文本
cleanchap = [ " ".join(m_cut(w)) for w in chapter.txt] 

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.998 seconds.
Prefix dict has been built succesfully.


In [7]:
# 将文本中的词语转换为词频矩阵  
from sklearn.feature_extraction.text import CountVectorizer

countvec = CountVectorizer(min_df = 5) 

wordmtx = countvec.fit_transform(cleanchap) 
wordmtx

<40x6535 sparse matrix of type '<class 'numpy.int64'>'
	with 72523 stored elements in Compressed Sparse Row format>

In [8]:
#基于词频矩阵X计算TF-IDF值  
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer()  
tfidf = transformer.fit_transform(wordmtx)  
tfidf

<40x6535 sparse matrix of type '<class 'numpy.float64'>'
	with 72523 stored elements in Compressed Sparse Row format>

In [9]:
# 设定LDA模型
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 10
ldamodel = LatentDirichletAllocation(n_components = n_topics)

In [10]:
# 拟合LDA模型
ldamodel.fit(wordmtx)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [11]:
# 拟合后模型的实质
print(ldamodel.components_.shape)
ldamodel.components_[:2]

(10, 6535)


array([[0.10000646, 0.1000067 , 1.14972784, ..., 0.10000487, 0.1       ,
        0.10000785],
       [0.1       , 0.1       , 0.1       , ..., 0.1       , 0.1       ,
        0.1       ]])

In [12]:
# 主题词打印函数
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i] 
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [13]:
n_top_words = 12
tf_feature_names = countvec.get_feature_names()
print_top_words(ldamodel, tf_feature_names, n_top_words)

Topic #0:
郭靖 欧阳克 周伯通 黄蓉 说道 武功 郭靖道 丘处机 弟子 杨康 洪七公 师父
Topic #1:
郭靖收 竟要 受挫 巧招 魔杖 最为 远远地 长嘶 脸面 狠劲 落脚 撕开
Topic #2:
郭靖 成吉思汗 欧阳锋 蒙古 大汗 鲁有脚 郭靖道 母亲 周伯通 拖雷 只见 小红马
Topic #3:
丘处机 铁木真 柯镇恶 一个 郭靖 完颜洪烈 朱聪 贫道 段天德 韩小莹 韩宝驹 彭连虎
Topic #4:
杨铁心 包惜弱 郭啸天 官兵 说道 丘处机 一个 金兵 杨二人 一声 丈夫 两人
Topic #5:
黄药师 洪七公 周伯通 欧阳锋 郭靖 欧阳克 说道 武功 兄弟 黄老邪 叫化 九阴真经
Topic #6:
黄药师 郭靖 黄蓉 师父 欧阳锋 柯镇恶 说道 一个 杨康 黄蓉道 一声 两人
Topic #7:
郭靖 黄蓉 完颜康 说道 裘千仞 陆冠英 师父 一个 王处一 侯通海 武功 不知
Topic #8:
黄蓉 郭靖 洪七公 欧阳锋 说道 师父 黄蓉道 武功 心中 两人 一个 欧阳克
Topic #9:
师父 郭靖 梅超风 铁木真 师哥 六怪 华筝 韩小莹 功夫 拖雷 说道 江南



## gensim实现

> class gensim.models.ldamodel.LdaModel(
>
>corpus = None : 用于训练模型的语料
>
>num_topics = 100 : 准备提取的主题数量
>
>id2word = None : 所使用的词条字典，便于结果阅读
>
>passes = 1 ：模型遍历语料库的次数，次数越多模型越精确，但是也更花时间
>
>)

用新出现的语料更新模型

    ldamodel.update(other_corpus)

gensim也提供了sklearn的API接口：sklearn_api.ldamodel，可以在sklearn中直接使用。

In [14]:
# 设定分词及清理停用词函数
# 熟悉Python的可以使用 open('stopWord.txt').readlines（） 获取停用词list，效率更高
stoplist = list(pd.read_csv('停用词.txt', names = ['w'], sep = 'aaa', 
                            encoding = 'utf-8', engine='python').w)
import jieba 
def m_cut(intxt):
    return [ w for w in jieba.cut(intxt) 
            if w not in stoplist and len(w) > 1 ] 

In [15]:
# 文档预处理，提取主题词  
chaplist = [m_cut(w) for w in chapter.txt]

In [16]:
# 生成文档对应的字典和bow稀疏向量
from gensim import corpora, models  

dictionary = corpora.Dictionary(chaplist)  
corpus = [dictionary.doc2bow(text) for text in chaplist] # 仍为list in list  

tfidf_model = models.TfidfModel(corpus) # 建立TF-IDF模型  
corpus_tfidf = tfidf_model[corpus] # 对所需文档计算TF-IDF结果
corpus_tfidf

In [17]:
from gensim.models.ldamodel import LdaModel

# 列出所消耗的时间备查
%time ldamodel = LdaModel(corpus, id2word = dictionary, \
                          num_topics = 10, passes = 2) 

Wall time: 2.27 s


***列出最重要的前若干个主题***

print_topics(num_topics=20, num_words=10)

In [18]:
ldamodel.print_topics()

[(0,
  '0.007*"郭靖" + 0.005*"黄蓉" + 0.003*"洪七公" + 0.003*"说道" + 0.003*"一个" + 0.003*"师父" + 0.002*"欧阳锋" + 0.002*"周伯通" + 0.002*"黄药师" + 0.002*"武功"'),
 (1,
  '0.012*"郭靖" + 0.007*"黄蓉" + 0.005*"说道" + 0.004*"一个" + 0.004*"欧阳锋" + 0.004*"师父" + 0.003*"洪七公" + 0.003*"柯镇恶" + 0.003*"梅超风" + 0.003*"武功"'),
 (2,
  '0.009*"郭靖" + 0.006*"黄蓉" + 0.003*"丘处机" + 0.003*"完颜康" + 0.003*"一个" + 0.003*"两人" + 0.002*"师父" + 0.002*"黄蓉道" + 0.002*"郭靖道" + 0.002*"不知"'),
 (3,
  '0.008*"郭靖" + 0.008*"黄蓉" + 0.004*"一个" + 0.003*"黄药师" + 0.003*"欧阳锋" + 0.003*"说道" + 0.003*"黄蓉道" + 0.003*"洪七公" + 0.003*"武功" + 0.002*"两人"'),
 (4,
  '0.010*"郭靖" + 0.008*"黄蓉" + 0.007*"欧阳锋" + 0.007*"洪七公" + 0.006*"师父" + 0.005*"周伯通" + 0.005*"说道" + 0.004*"黄药师" + 0.004*"武功" + 0.003*"一个"'),
 (5,
  '0.008*"黄蓉" + 0.006*"郭靖" + 0.003*"师父" + 0.003*"一个" + 0.003*"不知" + 0.003*"说道" + 0.003*"两人" + 0.002*"黄药师" + 0.002*"一声" + 0.002*"陆冠英"'),
 (6,
  '0.006*"郭靖" + 0.006*"黄药师" + 0.005*"黄蓉" + 0.004*"洪七公" + 0.004*"师父" + 0.003*"梅超风" + 0.003*"说道" + 0.003*"武功" + 0.003*"两人" + 0.003*"丘处机"'),
 

In [19]:
# 计算各语料的LDA模型值
corpus_lda = ldamodel[corpus_tfidf] # 此处应当使用和模型训练时相同类型的矩阵

for doc in corpus_lda:
    print(doc)

[(4, 0.97051376)]
[(8, 0.975271)]
[(1, 0.32965022), (4, 0.6413916)]
[(1, 0.97241247)]
[(1, 0.874216), (7, 0.100064695)]
[(1, 0.9701898)]
[(1, 0.9676881)]
[(1, 0.96999127)]
[(2, 0.8941156), (4, 0.08169592)]
[(4, 0.97942364)]
[(1, 0.9794066)]
[(1, 0.9784186)]
[(9, 0.9730704)]
[(1, 0.97571605)]
[(9, 0.9745509)]
[(7, 0.9777313)]
[(7, 0.9725293)]
[(4, 0.68615943), (9, 0.2945629)]
[(4, 0.97663146)]
[(4, 0.9713549)]
[(4, 0.9767679)]
[(4, 0.97859997)]
[(4, 0.97699827)]
[(5, 0.43091446), (9, 0.54037786)]
[(6, 0.9768866)]
[(1, 0.12405844), (9, 0.8556069)]
[(8, 0.97142935)]
[(1, 0.97340405)]
[(5, 0.97709596)]
[(3, 0.96511984)]
[(1, 0.9735802)]
[(2, 0.9743986)]
[(4, 0.9695384)]
[(8, 0.979204)]
[(1, 0.973753)]
[(1, 0.9775833)]
[(1, 0.9783438)]
[(1, 0.13982196), (4, 0.838614)]
[(9, 0.97785443)]
[(1, 0.97766405)]


In [20]:
ldamodel.get_topics()

array([[1.0723437e-05, 2.6136192e-03, 5.8189671e-05, ..., 1.0824628e-05,
        1.0236101e-05, 1.0833387e-05],
       [7.3880033e-06, 4.3376596e-03, 6.9850932e-05, ..., 8.3528821e-06,
        4.7353428e-06, 1.0846358e-05],
       [9.0887452e-06, 2.7104781e-03, 4.3433720e-05, ..., 9.1324737e-06,
        8.4627818e-06, 8.6944756e-06],
       ...,
       [8.9569430e-06, 3.3045991e-03, 1.0203754e-04, ..., 7.7200420e-06,
        7.6527076e-06, 7.0349661e-06],
       [9.0961430e-06, 3.3308326e-03, 7.2652030e-05, ..., 5.4732145e-06,
        5.8891233e-06, 5.5069972e-06],
       [4.4741637e-06, 3.0323293e-03, 8.7010703e-05, ..., 5.5355667e-06,
        6.8273484e-06, 4.4346125e-06]], dtype=float32)

In [21]:
# 检索和文本内容最接近的主题
query = chapter.txt[1] # 检索和第1章最接近的主题
query_bow = dictionary.doc2bow(m_cut(query)) # 频数向量
query_tfidf = tfidf_model[query_bow] # TF-IDF向量
print("转换后：", query_tfidf[:10])

ldamodel.get_document_topics(query_bow) # 需要输入和文档对应的bow向量

转换后： [(0, 0.007408473996700298), (2, 0.0022659955699091345), (3, 0.009122633472297354), (4, 0.0029774360320260706), (5, 0.003980155045506189), (6, 0.00640575498322018), (7, 0.009122633472297354), (8, 0.00043827823545052573), (9, 0.0009906731573462728), (10, 0.005694314521103244)]


[(4, 0.99987316)]

In [22]:
# 检索和文本内容最接近的主题
ldamodel[query_tfidf]

[(4, 0.97051376)]

## 结果的图形化呈现

pyLDAvis包引入自R，可以用交互式图形的方式呈现主题模型的分析结果。

同时支持sklearn和gensim包。

在许多系统配置下都会出现兼容问题。

In [27]:
# 对sklearn的LDA结果作呈现
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(ldamodel, tfidf, countvec)

AttributeError: 'LdaModel' object has no attribute 'components_'

In [26]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

AttributeError: module 'pyLDAvis' has no attribute 'gensim'

In [ ]:
pyLDAvis.disable_notebook() # 关闭notebook支持后，可以看到背后所生成的数据

## 实战练习

在其余参数全部固定不变的情况下，尝试分别用清理前矩阵、清理后原始矩阵、TF-IDF矩阵进行LDA模型拟合，比较分析结果。

在gensim拟合LDA时，分别将passes参数设置为1、5、10、50、100等，观察结果变化的情况，思考如何对该参数做最优设定。

请尝试对模型进行优化，得到对本案例较好的分析结果。

提示：使用gensim进行拟合更容易一些。